# Tests of cosmic homogeneity
This notebook is part of the *DoCrimes* project <https://github.com/davidwhogg/DoCrimes>.

## Authors
- **David W. Hogg** (NYU) (MPIA) (Flatiron)

## License
- This code is licensed for re-use under the open-source *MIT License*. See the file `LICENSE` for more information.

## To-do items and bugs:
- Actually use the jackknifes.
- Fix things such that `jack_and_repack()` doesn't need to do so much repacking.
- Make a quantitative estimate of the fractal dimension.
- Switch to a color scheme that is good in black-and-white.
- Figure out a way to estimate a "comoving volume" for the sample.

In [ ]:
import numpy as np
import pickle
import os
from scipy.spatial import KDTree
from astropy.io import fits
from astropy.io import ascii
from astropy.table import QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18
import pylab as plt

In [ ]:
# Set some standards.
clobber = False
rng = np.random.default_rng(17)
RECTFIG = [6., 3.5]
SQUAREFIG = [6., 6.]
dpi = 300

In [ ]:
# Read the Bright Star Catalog (don't forget to cite this!).
r = ascii.get_reader(ascii.Cds, readme="../data/bsc/ReadMe2") # horrifying file path
bsc_table = r.read("../data/bsc/catalog")
bright = bsc_table['Vmag'] < 4.
bsc_table = bsc_table[bright]
print(len(bsc_table))

In [ ]:
# Read the quasar catalog.
datafd = fits.open("../data/gaia_G20.0.fits") # terrible file path
data_table = QTable(datafd[1].data)
print(len(data_table))

In [ ]:
print(data_table.colnames)

In [ ]:
# Read the random catalog.
randfd = fits.open("../data/random_stardustm1064_G20.0_10x.fits") # upsetting file path
rand_table = QTable(randfd[1].data)
print(len(rand_table))

In [ ]:
# THIS IS A WRONG HACK. THERE IS A TRUE FACTOR, WHICH WE SHOULD USE.
factor = len(data_table) / len(rand_table)
print(factor)

In [ ]:
# Cut everything by the dust map.
ebvlim = 0.05
Id = data_table['ebv'] < ebvlim
data_table = data_table[Id]
Ir = rand_table['ebv'] < ebvlim
rand_table = rand_table[Ir]
print(len(data_table), len(rand_table))

In [ ]:
# Give l,b values to BSC.
bsc_table['ra']  = np.zeros_like(bsc_table['RAh']) + np.NaN
bsc_table['dec'] = np.zeros_like(bsc_table['RAh']) + np.NaN
bsc_table['l']   = np.zeros_like(bsc_table['RAh']) + np.NaN
bsc_table['b']   = np.zeros_like(bsc_table['RAh']) + np.NaN
for i in range(len(bsc_table)):
    rastr = "{:02d}h{:02d}m{:04.1f}s".format(bsc_table['RAh'][i],
                                         bsc_table['RAm'][i],
                                         bsc_table['RAs'][i])
    decstr = "{}{:02d}d{:02d}m{:02d}s".format(bsc_table['DE-'][i],
                                          bsc_table['DEd'][i],
                                          bsc_table['DEm'][i],
                                          bsc_table['DEs'][i])
    try:
        foo = SkyCoord(rastr, decstr, frame="icrs")
        bsc_table['ra'][i]  = foo.ra.to(u.deg).value
        bsc_table['dec'][i] = foo.dec.to(u.deg).value
        bsc_table['l'][i] = foo.galactic.l.to(u.deg).value
        bsc_table['b'][i] = foo.galactic.b.to(u.deg).value
    except:
        pass
print(len(bsc_table))

In [ ]:
# Overwrite l, b in the data catalog.
datac = SkyCoord(ra=data_table['ra']*u.degree, dec=data_table['dec']*u.degree, frame='icrs')
data_table['l'] = datac.galactic.l.to(u.deg).value
data_table['b'] = datac.galactic.b.to(u.deg).value
print(len(data_table))

In [ ]:
# Add l, b to the random catalog.
randc = SkyCoord(ra=rand_table['ra']*u.degree, dec=rand_table['dec']*u.degree, frame='icrs')
rand_table['l'] = randc.galactic.l.to(u.deg).value
rand_table['b'] = randc.galactic.b.to(u.deg).value
print(len(rand_table))

In [ ]:
# Make two hemsisphere samples, and plotting comparison samples.
dhi, dlo = data_table['b'] > 0., data_table['b'] < 0.
rhi, rlo = rand_table['b'] > 0., rand_table['b'] < 0.
plot_rhi, plot_rlo = rhi.copy(), rlo.copy()
plot_rhi[len(data_table):] = False
plot_rlo[len(data_table):] = False
print(np.sum(dhi), np.sum(dlo))

In [ ]:
def _bsc_size(bsc):
    return np.clip(4.5 - bsc['Vmag'], 0., None) ** 2.

def plot_bsc(ax, bsc):
    ax.scatter(bsc['ra'], bsc['dec'], color='k', marker='o', alpha=0.5,
               edgecolors="none",
               s = _bsc_size(bsc))

In [ ]:
# Plot the objects on the sky in (ra, dec).
scatterkwargs = {'marker':'o',
                 's': 0.3,
                 'alpha':0.5, 
                 'edgecolors': 'none',
                }

fig = plt.figure(figsize=RECTFIG)
fig.set_tight_layout(True)
for I in [dhi, dlo]:
    plt.scatter(data_table['ra'][I], data_table['dec'][I],
                **scatterkwargs)
plot_bsc(plt.gca(), bsc_table)
plt.xlim(360, 0)
plt.ylim(-90, 90)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("RA (deg)")
plt.ylabel("Dec (deg)")
plt.title("sky distribution")
plt.savefig("radec.png", dpi=dpi)

In [ ]:
# Don't do this.
if False:
    for I in [dhi, dlo]:
        plt.scatter(data_table['l'][I] * np.pi / 180.,
                    np.sin(data_table['b'][I] * np.pi / 180.),
                    **scatterkwargs)
    plt.ylim(-1, 1)
    plt.xlim(0, np.pi)
    plt.xlabel("Galactic longitude (rad)")
    plt.ylabel("sine of Galactic latitude")
    plt.title("data")
    plt.savefig("lb.png", dpi=dpi)

    plt.figure()
    for I in [plot_rhi, plot_rlo]:
        plt.scatter(rand_table['l'][I] * np.pi / 180.,
                    np.sin(rand_table['b'][I] * np.pi / 180.),
                    **scatterkwargs)
    plt.ylim(-1, 1)
    plt.xlim(0, np.pi)
    plt.xlabel("Galactic longitude (rad)")
    plt.ylabel("sine of Galactic latitude")
    plt.title("a sampling of the random catalog")
    plt.savefig("lb_random.png")

In [ ]:
def hogg_sky_plot(data, title, Is, bsc=None, **kwargs):
    """
    ## inputs:
    - `data`: table that must contain `l` longitudes in deg and `b` latitudes in deg.
    - `title`: plot title
    - `Is` : a list of boolean indices that index the data.
    - `**kwargs`: passed to scatter.
    
    ## bugs:
    - Should probably use astropy units.
    """
    xoff = np.sqrt(2.)
    def _project(ls, bs, xoff):
        thetas = ls
        rs = np.sqrt(2. - 2. * np.sin(np.abs(bs)))
        return (rs * np.cos(thetas) - xoff) * np.sign(bs) * -1., rs * np.sin(thetas)
    def _plot_grid(ax, xoff, fontdict=None):
        # make background
        ls1 = np.arange(0., 1.0001, 0.001) * np.pi
        ls2 = 2. * np.pi - ls1
        bs = np.zeros_like(ls1) + 0.0001 * np.pi
        for sign in (1., -1.):
            xs, ys1 = _project(ls1, sign * bs, xoff)
            _,  ys2 = _project(ls2, sign * bs, xoff)
            ax.fill_between(xs, ys1, ys2, color="w", zorder=-np.Inf)
        # make l grid
        bs = np.array([0.5, 0.1, 0.0001]) * np.pi
        for l in np.arange(0, 359, 30).astype(int):
            ls = np.zeros_like(bs) + l * np.pi / 180.
            for sign in (1., -1.):
                xs, ys = _project(ls, sign * bs, xoff)
                ax.plot(xs, ys, "k-", lw=0.5, alpha=0.5)
                if fontdict is not None:
                    lstr = "{}".format(l)
                    ax.text(xs[-2], ys[-2], lstr, fontdict=fontdict)
        # make b grid
        ls = np.arange(0., 360.01, 1.) * np.pi / 180.
        for b in (10, 30, 50, 70):
            bs = np.zeros_like(ls) + b * np.pi / 180.
            for sign in (1., -1.):
                xs, ys = _project(ls, sign * bs, xoff)
                ax.plot(xs, ys, "k-", lw=0.5, alpha=0.5)
                if fontdict is not None:
                    bstr = "{}".format(b)
                    ax.text(xs[285], ys[285], bstr, fontdict=fontdict)
    def _plot_bsc(ax, xoff, bsc):
        xs, ys = _project(bsc['l'] * np.pi / 180.,
                          bsc['b'] * np.pi / 180., xoff)
        ax.scatter(xs, ys, color='k', marker='o', alpha=0.5,
                   edgecolors="none",
                   s = _bsc_size(bsc))
    xs, ys = _project(data['l'] * np.pi / 180.,
                      data['b'] * np.pi / 180., xoff)
    fig = plt.figure(figsize=RECTFIG)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    for I in Is:
        ax.scatter(xs[I], ys[I], **kwargs)
    labeldict = {'fontsize': 8,
                 'fontweight': 1,
                 'verticalalignment': 'center',
                 'horizontalalignment': 'center',
                 'alpha': 0.5}
    _plot_grid(ax, xoff, labeldict)
    if bsc is not None:
        _plot_bsc(ax, xoff, bsc)
    ax.set_xlim(-2 * xoff, 2 * xoff)
    ax.set_ylim(-xoff, xoff)
    ax.set_aspect('equal', adjustable='box')
    titledict = {'fontsize': plt.rcParams['axes.titlesize'],
                 'fontweight': plt.rcParams['axes.titleweight'],
                 'verticalalignment': 'top',
                 'horizontalalignment': 'center'}
    ax.text(0., 1.35, title, fontdict=titledict)
    return fig

In [ ]:
# Plot the objects on the sky in (l, b).
fig = hogg_sky_plot(data_table, "data",
                    [dhi, dlo], bsc=bsc_table, **scatterkwargs)
fig.savefig("lb_better.png", dpi=dpi, bbox_inches="tight")
fig = hogg_sky_plot(rand_table, "a sampling of the random catalog",
                    [plot_rhi, plot_rlo], bsc=bsc_table, **scatterkwargs)
fig.savefig("lb_random_better.png", dpi=dpi, bbox_inches="tight")

In [ ]:
# Make the redshift histograms.
bins = np.arange(-0.1, 4.601, 0.1)
bincntrs = 0.5 * (bins[1:] + bins[:-1])
binwids = (bins[1:] - bins[:-1])
nzhi, _ = np.histogram(data_table['redshift_spz'][dhi], bins)
nzhi = nzhi.astype(float) / (np.sum(rhi) * factor) / binwids
nzlo, _ = np.histogram(data_table['redshift_spz'][dlo], bins)
nzlo = nzlo.astype(float) / (np.sum(rlo) * factor) / binwids

In [ ]:
# Plot redshift histograms.
stepkwargs = {'alpha': 0.8,
              'where': 'mid',
             }

fig = plt.figure(figsize=SQUAREFIG)
fig.set_tight_layout(True)
plt.axhline(0., color="k", lw = 0.5, alpha=0.5)
plt.step(bincntrs, nzhi, label="NGC", **stepkwargs)
plt.step(bincntrs, nzlo, label="SGC", **stepkwargs)
plt.legend()
plt.xlim(0., 4.5)
plt.xlabel("spz redshift")
plt.ylabel("fraction per unit redshift")
plt.title("redshift distribution")
plt.savefig("zhist.png", dpi=dpi)

In [ ]:
# Give the random points redshifts.
rand_table['redshift_spz'] = rng.choice(data_table['redshift_spz'],
                                        size=len(rand_table), replace=True)

In [ ]:
# Get comoving LOS distances.
# This could be sped up if we noted that many redshifts are repeated!
data_table['comoving_distance'] = Planck18.comoving_distance(data_table['redshift_spz']).value
rand_table['comoving_distance'] = Planck18.comoving_distance(rand_table['redshift_spz']).value

In [ ]:
# Cartesian positions in comoving space.
def add_xyz(table):
    r = table['comoving_distance']
    cb = np.cos(table['b'] * np.pi / 180.)
    x = r * cb * np.cos(table['l'] * np.pi / 180.)
    y = r * cb * np.sin(table['l'] * np.pi / 180.)
    z = r * np.sin(table['b'] * np.pi / 180.)
    table['xyz'] = np.array([x, y, z]).T
    return

add_xyz(data_table)
add_xyz(rand_table)

In [ ]:
# Make cartesian plots.
fig = plt.figure(figsize=SQUAREFIG)
fig.set_tight_layout(True)
for I in [dhi, dlo]:
    plt.scatter(data_table['xyz'][I,0], data_table['xyz'][I,2],
                **scatterkwargs)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(-5999, 5999)
plt.ylim(-7000, 7000)
plt.xlabel("comoving x (Mpc)")
plt.ylabel("comoving z (Mpc)")
plt.savefig("cartesianxz.png", dpi=dpi)

fig = plt.figure(figsize=SQUAREFIG)
fig.set_tight_layout(True)
for I in [dhi, dlo]:
    plt.scatter(data_table['xyz'][I,1], data_table['xyz'][I,2],
                **scatterkwargs)
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(-5999, 5999)
plt.ylim(-7000, 7000)
plt.xlabel("comoving y (Mpc)")
plt.ylabel("comoving z (Mpc)")
plt.savefig("cartesianyz.png", dpi=dpi)

In [ ]:
def all_pair_counts(data_table, rand_table, factor):

    # Make subsample indices.
    dhi = data_table['b'] > 0.
    dlo = data_table['b'] < 0.
    rhi = rand_table['b'] > 0.
    rlo = rand_table['b'] < 0.

    # Make trees in preparation for counting pairs.
    print("making trees...")
    data_tree = KDTree(data_table['xyz'])
    dhi_tree = KDTree(data_table['xyz'][dhi])
    dlo_tree = KDTree(data_table['xyz'][dlo])
    rand_tree = KDTree(rand_table['xyz'])
    rhi_tree = KDTree(rand_table['xyz'][rhi])
    rlo_tree = KDTree(rand_table['xyz'][rlo])

    # Make the DD counts (cumulative) out to various radii.
    # Note the controversial `- N` adjustments to the self-counts.
    rs = np.exp(np.arange(np.log(5.), np.log(1000.), 0.25))
    print("making DD_hi...")
    DD_hi = (dhi_tree.count_neighbors(data_tree, rs).astype(float)
             - np.sum(dhi)) # controversial
    print("making DD_lo...")
    DD_lo = (dlo_tree.count_neighbors(data_tree, rs).astype(float)
             - np.sum(dlo)) # controversial

    # Make the DR and RD counts.
    print("making DR_hi...")
    DR_hi = factor * dhi_tree.count_neighbors(rand_tree, rs).astype(float)
    print("making DR_lo...")
    DR_lo = factor * dlo_tree.count_neighbors(rand_tree, rs).astype(float)
    print("making RD_hi...")
    RD_hi = factor * rhi_tree.count_neighbors(data_tree, rs).astype(float)
    print("making RD_lo...")
    RD_lo = factor * rlo_tree.count_neighbors(data_tree, rs).astype(float)

    # Make the RR counts.
    print("making RR_hi...")
    RR_hi = factor ** 2 * (rhi_tree.count_neighbors(rand_tree, rs).astype(float)
                           - np.sum(rhi)) # controversial
    print("making RR_lo...")
    RR_lo = factor ** 2 * (rlo_tree.count_neighbors(rand_tree, rs).astype(float)
                           - np.sum(rlo)) # controversial

    print("done.")
    return rs, np.array([DD_hi, DD_lo]), \
               np.array([DR_hi, DR_lo]), \
               np.array([RD_hi, RD_lo]), \
               np.array([RR_hi, RR_lo])

In [ ]:
# Count all pairs.
fn = "counts.pkl"
rs = None
if os.path.exists(fn):
    with open(fn, "rb") as fd:
        rs, DD, DR, RD, RR = pickle.load(fd)
if rs is None or clobber:
    rs, DD, DR, RD, RR = all_pair_counts(data_table, rand_table, factor)
    with open(fn, "wb") as fd:
        pickle.dump((rs, DD, DR, RD, RR), fd)
print(rs.shape, DD.shape, RR.shape)

In [ ]:
def jack_and_repack(data_table, rand_table, factor):
    DD = []
    DR = []
    RD = []
    RR = []

    dell = 30. # deg
    for ell1 in np.arange(0., 359., dell):
        ell2 = ell1 + dell
        dI = np.logical_or(data_table['l'] < ell1, data_table['l'] >= ell2)
        rI = np.logical_or(rand_table['l'] < ell1, rand_table['l'] >= ell2)
        print("working on", ell1, ell2, np.sum(dI), np.sum(rI))

        _, DD1, DR1, RD1, RR1 = all_pair_counts(data_table[dI], rand_table[rI], factor)

        DD += [DD1, ]
        print(np.array(DD).shape)
        DR += [DR1, ]
        RD += [RD1, ]
        RR += [RR1, ]

    return np.array(DD), np.array(DR), np.array(RD), np.array(RR)

In [ ]:
# Jackknife the shit out of everything.
fn = "jacks.pkl"
DD_jack = None
if os.path.exists(fn):
    with open(fn, "rb") as fd:
        DD_jack, DR_jack, RD_jack, RR_jack \
        = pickle.load(fd)
if DD_jack is None or clobber:
    DD_jack, DR_jack, RD_jack, RR_jack \
        = jack_and_repack(data_table, rand_table, factor)
    with open(fn, "wb") as fd:
        pickle.dump((DD_jack, DR_jack, RD_jack, RR_jack),
                    fd)
print(rs.shape, DD_jack.shape, DR_jack.shape, RD_jack.shape, RR_jack.shape)

In [ ]:
# Make DD and put an error bar on it!
njack = len(DD_jack)
DDall = np.sum(DD, axis=0)
DD_jack_mean = np.mean(DD_jack, axis=0)[None, :, :]
DDall_var = ((njack - 1.) / njack) * np.sum(np.sum(DD_jack -
                                                   DD_jack_mean,
                                                   axis=1) ** 2,
                                            axis=0)
DDall_err = np.sqrt(DDall_var)
print(DDall.shape, DDall_err.shape, DDall_err)

In [ ]:
# Plot the fractal dimension.
plotkwargs = {'linestyle': 'none', 'marker': 'o', 'alpha': 0.8, }

fig = plt.figure(figsize=SQUAREFIG)
fig.set_tight_layout(True)
plt.errorbar(rs, DDall, yerr=DDall_err, color="k", **plotkwargs)
plt.loglog()
plt.xlabel("maximum comoving pair separation (Mpc)")
plt.ylabel("cumulative number of pairs")
plt.title("pair counts")
plt.savefig("cumulativeDD.png", dpi=dpi)

In [ ]:
# Make DD ratio to an expectation and put an error bar on it!
DDoverexp = np.sum(DD, axis=0) / np.sum(DR + RD - RR, axis=0)
DDoverexp_jack = np.sum(DD_jack, axis=1) / np.sum(DR_jack +
                                                  RD_jack -
                                                  RR_jack,
                                                  axis=1)
DDoverexp_mean = np.mean(DDoverexp_jack, axis=0)[None, :]
DDoverexp_var = ((njack - 1.) / njack) * np.sum((DDoverexp_jack -
                                                 DDoverexp_mean) ** 2,
                                                axis=0)
DDoverexp_err = np.sqrt(DDoverexp_var)
print(DDoverexp.shape, DDoverexp_jack.shape, DDoverexp_err.shape)

In [ ]:
# Plot the difference from the D=3 expectation
fig, axes = plt.subplots(2, 1, sharex=True, figsize=SQUAREFIG)
fig.set_tight_layout(True)
uniform = rs ** 3
ax = axes[0]
ax.errorbar(rs, DDall, yerr=DDall_err, color="k", **plotkwargs)
ax.loglog()
ax.set_ylabel("cumulative number of pairs")
ax.set_title("pair counts")
ax = axes[1]
ax.axhline(1, color="k", lw=1, alpha=0.5)
ax.errorbar(rs, DDoverexp, yerr=DDoverexp_err, color="k", **plotkwargs)
ax.semilogx()
ax.set_xlabel("maximum comoving pair separation (Mpc)")
ax.set_ylabel("number relative to expectation")
plt.savefig("cumulativeDD_DR.png", dpi=dpi)

In [ ]:
# Now turn the cumulative counts into differential counts.
dDD = DD[:, 1:] - DD[:, :-1]
dDD_jack = DD_jack[:, :, 1:] - DD_jack[:, :, :-1]
d2DR = DR[:, 1:] - DR[:, :-1] \
     + RD[:, 1:] - RD[:, :-1]
d2DR_jack = DR_jack[:, :, 1:] - DR_jack[:, :, :-1] \
          + RD_jack[:, :, 1:] - RD_jack[:, :, :-1]
dRR = RR[:, 1:] - RR[:, :-1]
dRR_jack = RR_jack[:, :, 1:] - RR_jack[:, :, :-1]
rcs = np.exp(0.5 * (np.log(rs[1:]) + np.log(rs[:-1])))
print(rcs.shape, d2DR.shape, d2DR_jack.shape)

In [ ]:
# Estimate the correlation function.
LS = (dDD - d2DR + dRR) / dRR
LS_jack = (dDD_jack - d2DR_jack + dRR_jack) / dRR_jack
LS_mean = np.mean(LS_jack, axis=0)[None, :, :]
LS_var = np.sum((LS_jack - LS_mean) ** 2, axis=0)
LS_err = np.sqrt(LS_var)
print(LS.shape, LS_jack.shape, LS_mean.shape, LS_err.shape)

In [ ]:
# Plot the correlation function.
fig = plt.figure(figsize=SQUAREFIG)
fig.set_tight_layout(True)
plt.axhline(0., color="k", lw=0.5, alpha=0.5)
plt.errorbar(rcs, LS[0, :], yerr=LS_err[0, :], label="NGC", **plotkwargs)
plt.errorbar(rcs, LS[1, :], yerr=LS_err[1, :], label="SGC", **plotkwargs)
plt.loglog()
plt.legend()
plt.xlim(5., 1000.)
plt.xlabel("comoving pair separation (Mpc)")
plt.ylabel("L-S correlation-function estimate")
plt.title("auto-correlation function")
plt.savefig("corrfunc.png", dpi=dpi)